# imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import Xception, DenseNet201
from tensorflow.keras.callbacks import LearningRateScheduler
import random
from PIL import Image
import shutil

# global variables

In [25]:
# image size options: 192, 224, 311, 512
IMAGE_DIMENSION = 192
VECTOR_LEN = IMAGE_DIMENSION**2
NUM_CLASS = 96

train_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train'
val_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val'
test_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test'

BATCH_SIZE = 10
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

# eda

# generate datasets

In [3]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11459 images belonging to 104 classes.
Found 3710 images belonging to 104 classes.
Found 1294 images belonging to 104 classes.


### defining functions

In [4]:
def get_subfolder_list(source_folder):
    # returns a list of all the subfolders in the source_folder
    class_list_dir = []

    for file in os.listdir(source_folder):
        d = os.path.join(source_folder, file)
        if os.path.isdir(d):
            class_list_dir.append(d)

    return class_list_dir

def map_classes(subfolder_list):
    # returns a dict with the flower as the key, and (count, directory) as the values
    class_dict = {}

    for class_folder in subfolder_list:
        file_count = sum(len(files) for _, _, files in os.walk(class_folder))
        class_dict[class_folder[24:]] = file_count, class_folder
        
    return class_dict

def get_flower_count(class_dict):
    # returns a list of the number of flowers found in the dict
    flower_count = []

    for flower in list(class_dict.values()):
        flower_count.append(flower[0])

    return flower_count

In [5]:
def get_metrics(flower_count):
    # returns basic metrics when given a list of flower value count
    class_std = np.std(flower_count)
    class_max = max(flower_count)
    class_min = min(flower_count)
    class_mean = np.mean(flower_count)
    class_first_quartile = np.percentile(flower_count, 25)
    class_third_quartile = np.percentile(flower_count, 75)
    class_tenth_percentile = np.percentile(flower_count, 10)
    class_fifth_percentile = np.percentile(flower_count, 5)

    print(f'0. standard deviation: {class_std}')
    print(f'1. max: {class_max}')
    print(f'2. min: {class_min}')
    print(f'3. mean: {class_mean}')
    print(f'4. 25%: {class_first_quartile}')
    print(f'5. 75%: {class_third_quartile}')
    print(f'6. 10%: {class_tenth_percentile}')
    print(f'7. 5%: {class_fifth_percentile}')
    
    return (class_std, class_max, class_min, class_mean, class_first_quartile,
class_third_quartile, class_tenth_percentile, class_fifth_percentile)

### testing above functions / eda

In [6]:
train_subfolders = get_subfolder_list(train_dir)
train_dict = map_classes(train_subfolders)
train_flower_count = get_flower_count(train_dict)

In [7]:
train_subfolders[:5]

['data/jpeg-192x192/train/toad lily',
 'data/jpeg-192x192/train/love in the mist',
 'data/jpeg-192x192/train/monkshood',
 'data/jpeg-192x192/train/azalea',
 'data/jpeg-192x192/train/fritillary']

In [8]:
get_metrics(train_flower_count)

0. standard deviation: 132.99130714962862
1. max: 707
2. min: 16
3. mean: 111.1826923076923
4. 25%: 30.5
5. 75%: 113.5
6. 10%: 19.0
7. 5%: 17.15


(132.99130714962862, 707, 16, 111.1826923076923, 30.5, 113.5, 19.0, 17.15)

In [9]:
val_subfolders = get_subfolder_list(val_dir)
val_dict = map_classes(val_subfolders)
val_flower_count = get_flower_count(val_dict)

In [10]:
get_metrics(val_flower_count)

0. standard deviation: 42.990624407913046
1. max: 228
2. min: 5
3. mean: 35.69230769230769
4. 25%: 9.0
5. 75%: 37.0
6. 10%: 6.0
7. 5%: 6.0


(42.990624407913046, 228, 5, 35.69230769230769, 9.0, 37.0, 6.0, 6.0)

# preprocessing

### more functions

In [17]:
def copy_subfolder(source):
    # copies source folder to a new directory with '_new' attached to the end of the folder name
    prev_dir_index = source.rfind('/')
    destination = source[:prev_dir_index] + '_new' + source[prev_dir_index:]
    
    
    if not os.path.exists(destination):
        result = shutil.copytree(source, destination, symlinks=False, ignore=None, 
                                 copy_function=shutil.copy2, ignore_dangling_symlinks=False, 
                                 dirs_exist_ok=False)
    else:
        print(f'{destination} already exists')
        
    return result
    
def item_count(folder):
    # helper function to identify which folders to remove
    file_count = sum(len(files) for _, _, files in os.walk(folder))
    
    return file_count
    
def get_short_list(subfolder_list, n):
    # find a list of classes that are divided by the specified n value
    temp_dict = map_classes(subfolder_list)
    temp_flower_count = get_flower_count(temp_dict)
    
    move_list = []
    ignore_list = []
    
    percent_cutoff = np.percentile(temp_flower_count, n)
    
    for subfolder in subfolder_list:
        if item_count(subfolder) >= percent_cutoff:
            move_list.append(subfolder)
        else:
            ignore_list.append(subfolder)
            
    return move_list, ignore_list
    
def trim(subfolder_list):
    # main function used to copy classes into new train, val, test 
    for subfolder in subfolder_list:
        copy_subfolder(subfolder)

        val_subfolder = subfolder.replace('/train/', '/val/')
        copy_subfolder(val_subfolder)

        test_subfolder = subfolder.replace('/train/', '/test/')
        copy_subfolder(test_subfolder)

In [12]:
move_list, ignore_list = get_short_list(train_subfolders, 10)

In [13]:
move_list[:5]

['data/jpeg-192x192/train/toad lily',
 'data/jpeg-192x192/train/love in the mist',
 'data/jpeg-192x192/train/monkshood',
 'data/jpeg-192x192/train/azalea',
 'data/jpeg-192x192/train/fritillary']

In [ ]:
trim(move_list)

In [64]:
src = 'data/jpeg-192x192/train/iris'

copy_subfolder(src)

data/jpeg-192x192/train_new/iris
23


FileExistsError: [Errno 17] File exists: 'data/jpeg-192x192/train_new/iris'

In [29]:
# updating the directories as new folders are made 
train_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train_new'
val_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val_new'
test_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test_new'

# generate datasets

In [19]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


# modeling

In [20]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
lr_callback = LearningRateScheduler(scheduler)

In [32]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [33]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,4)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5, input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 23, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 45632)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 45632)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

In [34]:
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1133/1133 [==============================] - ETA: 0s - loss: 4.1504 - accuracy: 0.0831WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x2aa828430> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOS

### simple model

In [35]:
model = models.Sequential()
model.add(layers.Conv2D(filters=64, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 192, 192, 64)      832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 96, 96, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 589824)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 96)                56623200  
_________________________________________________________________
activation_3 (Activation)    (None, 96)                0         
Total params: 56,624,032
Trainable params: 56,624,032
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux

KeyboardInterrupt: 

### simple model 1

In [36]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 96)                28311648  
_________________________________________________________________
activation_4 (Activation)    (None, 96)                0         
Total params: 28,312,064
Trainable params: 28,312,064
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux

KeyboardInterrupt: 

### simple 2

In [37]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (2,2), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 95, 95, 16)        2064      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 47, 47, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 35344)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 96)                3393120   
_________________________________________________________________
activation_5 (Activation)    (None, 96)               

### simple 3

In [38]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 96)                1476192   
_________________________________________________________________
activation_6 (Activation)    (None, 96)               

### simple 4 - batch size increase

In [39]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [40]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [41]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 96)                1476192   
_________________________________________________________________
activation_7 (Activation)    (None, 96)               